# Assignment 2: Milestone I Natural Language Processing
## Task 2&3
#### Student Name: Ngoc Khanh Linh Nguyen
#### Student ID: 3927588

Date: 27/09/2023

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: please include all the libraries you used in your assignment, e.g.,:
* pandas
* re
* numpy

## Introduction
You should give a brief information of this assessment task here.

<span style="color: red"> Note that this is a sample notebook only. You will need to fill in the proper markdown and code blocks. You might also want to make necessary changes to the structure to meet your own needs. Note also that any generic comments written in this notebook are to be removed and replace with your own words.</span>

## Importing libraries 

In [1]:
# Code to import libraries as you need in this assessment, e.g.,
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import re
from nltk.probability import *
from sklearn.model_selection import KFold
%store -r tk_descriptions

In [2]:
len(tk_descriptions)

776

## Task 2. Generating Feature Representations for Job Advertisement Descriptions

...... Sections and code blocks on buidling different document feature represetations


<span style="color: red"> You might have complex notebook structure in this section, please feel free to create your own notebook structure. </span>

In [3]:
# an example of job description to investigate throughout the task
emp = 11

In [4]:
# read the vocab saved from task 1
vocabFile = "./vocab.txt"
with open(vocabFile, 'r') as f:
    vocab = f.read().splitlines()
    
vocab

['aap: 0',
 'aaron: 1',
 'aat: 2',
 'abb: 3',
 'abenefit: 4',
 'aberdeen: 5',
 'abi: 6',
 'abilities: 7',
 'abreast: 8',
 'abroad: 9',
 'absence: 10',
 'absolute: 11',
 'ac: 12',
 'aca: 13',
 'academic: 14',
 'academy: 15',
 'acca: 16',
 'accept: 17',
 'acceptable: 18',
 'acceptance: 19',
 'accepted: 20',
 'access: 21',
 'accessible: 22',
 'accident: 23',
 'accommodates: 24',
 'accommodation: 25',
 'accomplished: 26',
 'accordance: 27',
 'account: 28',
 'accountabilities: 29',
 'accountability: 30',
 'accountable: 31',
 'accountancy: 32',
 'accountant: 33',
 'accountants: 34',
 'accounting: 35',
 'accounts: 36',
 'accreditation: 37',
 'accredited: 38',
 'accruals: 39',
 'accuracy: 40',
 'accurate: 41',
 'accurately: 42',
 'achievable: 43',
 'achieve: 44',
 'achieved: 45',
 'achievement: 46',
 'achievements: 47',
 'achiever: 48',
 'achieving: 49',
 'acii: 50',
 'acquired: 51',
 'acquisition: 52',
 'acquisitions: 53',
 'act: 54',
 'acting: 55',
 'action: 56',
 'actions: 57',
 'actionscri

In [5]:
print(len(vocab))

vocab = [element.split(':')[0] for element in vocab] # extract the word only from vocab.txt 
print(len(vocab))


5175
5175


In [6]:
vocab[emp]

'absolute'

### Generate Count vector representation

In [7]:
cVectorizer = CountVectorizer(analyzer="word", vocabulary=vocab) # initialise the CountVectorizer

In [8]:
count_features = cVectorizer.fit_transform([' '.join(des) for des in tk_descriptions]) # generate the count vector representation for all descriptions
print(count_features.shape)

(776, 5175)


In [9]:
# investigate the count_features
count_features[emp]

<1x5175 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

In [10]:
# read webIndex saved from text file
with open("webIndex.txt", 'r') as file: 
    webIndex = file.read().splitlines()
    
webIndex[emp]

'70757932'

In [11]:
# validator function 
def validator(data_features, vocab, j_ind, webindex,description_txts,tokenised_descriptions):
    print("Web Index:", webIndex[j_ind]) # print out the Jobdescription ID
    print("--------------------------------------------")
    print("txt:",description_txts[j_ind]) # print out the txt of the job description
    print("Description tokens:",tokenised_descriptions[j_ind]) # print out the tokens of the job description
    print("--------------------------------------------\n")
    print("Vector representation:\n") # printing the vector representation as format 'word:value' (
                                      # the value is an integer for count vector; and a float value for tfidf

    for word, value in zip(vocab, data_features.toarray()[j_ind]): 
        if value > 0:
            print(str(vocab.index(word))+":"+str(value), end =', ')

In [12]:
# recall the job_data list saved from task 1
%store -r job_data

In [13]:
job_data['data'][emp] # investigate the example job data

b'Title: European Payroll\nWebindex: 70757932\nCompany: Brookstreet UK\nDescription: Well established company based in Hemel Hempstead looking for someone to join the payroll team to process the payroll for their European employees Have you previously worked within a role where you have been processing European payroll? Are you available long term? Are you happy to work full time hours? This position will be Monday to Friday working ****:30am to ****:30pm. The role will be working within the payroll team to process payroll for their European employees. This is a temporary to permanent position so you must be available long term'

In [14]:
validator(count_features, vocab, emp, webIndex, job_data['data'], tk_descriptions)

Web Index: 70757932
--------------------------------------------
txt: b'Title: European Payroll\nWebindex: 70757932\nCompany: Brookstreet UK\nDescription: Well established company based in Hemel Hempstead looking for someone to join the payroll team to process the payroll for their European employees Have you previously worked within a role where you have been processing European payroll? Are you available long term? Are you happy to work full time hours? This position will be Monday to Friday working ****:30am to ****:30pm. The role will be working within the payroll team to process payroll for their European employees. This is a temporary to permanent position so you must be available long term'
Description tokens: ['established', 'hemel', 'hempstead', 'payroll', 'process', 'payroll', 'european', 'employees', 'previously', 'worked', 'processing', 'european', 'payroll', 'long', 'term', 'happy', 'time', 'hours', 'monday', 'friday', 'pm', 'payroll', 'process', 'payroll', 'european', 'em

In [15]:
# function to represent count vector as required format
def vectorRepresentation(data_features, webIndex, vocab, emp):
    webIndex = "#" + webIndex[emp]
    count = []
    for word, value in zip(vocab, data_features.toarray()[emp]):
        if value > 0:
            count.append(str(vocab.index(word)) + ":" + str(value))
    rep = webIndex + "," + ",".join(ele for ele in count)
    return rep

In [16]:
vectorRepresentation(count_features, webIndex, vocab, emp)

'#70757932,1562:2,1662:1,1683:3,1953:1,2123:1,2170:1,2171:1,2243:1,2753:2,3008:1,3395:5,3432:1,3514:1,3621:1,3664:2,3667:1,4706:1,4713:2,4758:1,5124:1'

In [17]:
len(webIndex)

776

In [18]:
len(tk_descriptions)

776

In [19]:
# save count vector in required format in countVectorList
countVectorList = [vectorRepresentation(count_features, webIndex, vocab, i) for i in range(len(tk_descriptions))]

In [20]:
len(countVectorList)

776

In [21]:
countVectorList[emp]

'#70757932,1562:2,1662:1,1683:3,1953:1,2123:1,2170:1,2171:1,2243:1,2753:2,3008:1,3395:5,3432:1,3514:1,3621:1,3664:2,3667:1,4706:1,4713:2,4758:1,5124:1'

#### Saving outputs
Save the count vector representation as per spectification.
- count_vectors.txt

In [22]:
# code to save count vector in required format to 'count_vectors.txt'
def saveCountVector(countFile, countVectorList):
    outfile = open(countFile, 'w')
    string = "\n".join(ele for ele in countVectorList)
    outfile.write(string)
    outfile.close()

In [23]:
saveCountVector("count_vectors.txt", countVectorList)

### Using In-house trained Word2vec model to generate vector representation

#### Unweighted vector representation for each job description

In [24]:
import gensim.models
sentences = tk_descriptions
# train my own model using Word2vec
description_w2v = gensim.models.Word2Vec(sentences=sentences, vector_size=100)

In [25]:
print(description_w2v)

Word2Vec<vocab=2745, vector_size=100, alpha=0.025>


In [26]:
# extract KeyedVectors object from the trained model
desw2v_wv = description_w2v.wv

In [27]:
len(desw2v_wv) # length of the vocabulary 

2745

In [28]:
# function to generate document embeddings based on word embeddings
def docvecs(embeddings, docs):
    vecs = np.zeros((len(docs), embeddings.vector_size))
    for i, doc in enumerate(docs):
        valid_keys = [term for term in doc if term in embeddings.key_to_index]
        docvec = np.vstack([embeddings[term] for term in valid_keys])
        """
        Note: using `sum` here, other 'pooling' options are possible too,
        e.g. mean, etc.
        """
        docvec = np.sum(docvec, axis=0)
        vecs[i,:] = docvec
    return vecs

In [29]:
# generate document embeddings 
descriptionW2v_dvs = docvecs(desw2v_wv, tk_descriptions)

#### Weighted (TF-IDF Weighted) vector representation

In [30]:
# generate tf-idf vectors
from sklearn.feature_extraction.text import TfidfVectorizer
tVectorizer = TfidfVectorizer(analyzer = "word",vocabulary = [word.split(': ')[0] for word in vocab]) # initialised the TfidfVectorizer
tfidf_features = tVectorizer.fit_transform([' '.join(des) for des in tk_descriptions]) # generate the tfidf vector representation for all articles

In [31]:
tfidf_features

<776x5175 sparse matrix of type '<class 'numpy.float64'>'
	with 62323 stored elements in Compressed Sparse Row format>

In [32]:
validator(tfidf_features,vocab, emp, webIndex, job_data['data'],tk_descriptions)
# validator(count_features, vocab, emp, webIndex, job_data['data'], tk_descriptions)

Web Index: 70757932
--------------------------------------------
txt: b'Title: European Payroll\nWebindex: 70757932\nCompany: Brookstreet UK\nDescription: Well established company based in Hemel Hempstead looking for someone to join the payroll team to process the payroll for their European employees Have you previously worked within a role where you have been processing European payroll? Are you available long term? Are you happy to work full time hours? This position will be Monday to Friday working ****:30am to ****:30pm. The role will be working within the payroll team to process payroll for their European employees. This is a temporary to permanent position so you must be available long term'
Description tokens: ['established', 'hemel', 'hempstead', 'payroll', 'process', 'payroll', 'european', 'employees', 'previously', 'worked', 'processing', 'european', 'payroll', 'long', 'term', 'happy', 'time', 'hours', 'monday', 'friday', 'pm', 'payroll', 'process', 'payroll', 'european', 'em

In [33]:
# write tf-idf vectors to file
def write_vectorFile(data_features,filename):
    num = data_features.shape[0] # the number of document
    out_file = open(filename, 'w') # creates a txt file and open to save the vector representation
    for a_ind in range(0, num): # loop through each article by index
        for f_ind in data_features[a_ind].nonzero()[1]: # for each word index that has non-zero entry in the data_feature
            value = data_features[a_ind][0,f_ind] # retrieve the value of the entry from data_features
            out_file.write("{}:{} ".format(f_ind,value)) # write the entry to the file in the format of word_index:value
        out_file.write('\n') # start a new line after each article
    out_file.close() # close the file

In [34]:
write_vectorFile(tfidf_features, 'description_tVector.txt')

In [35]:
vocabFile = "./vocab.txt"
with open(vocabFile, 'r') as f:
    vocab = f.read().splitlines()

In [36]:
vocab

['aap: 0',
 'aaron: 1',
 'aat: 2',
 'abb: 3',
 'abenefit: 4',
 'aberdeen: 5',
 'abi: 6',
 'abilities: 7',
 'abreast: 8',
 'abroad: 9',
 'absence: 10',
 'absolute: 11',
 'ac: 12',
 'aca: 13',
 'academic: 14',
 'academy: 15',
 'acca: 16',
 'accept: 17',
 'acceptable: 18',
 'acceptance: 19',
 'accepted: 20',
 'access: 21',
 'accessible: 22',
 'accident: 23',
 'accommodates: 24',
 'accommodation: 25',
 'accomplished: 26',
 'accordance: 27',
 'account: 28',
 'accountabilities: 29',
 'accountability: 30',
 'accountable: 31',
 'accountancy: 32',
 'accountant: 33',
 'accountants: 34',
 'accounting: 35',
 'accounts: 36',
 'accreditation: 37',
 'accredited: 38',
 'accruals: 39',
 'accuracy: 40',
 'accurate: 41',
 'accurately: 42',
 'achievable: 43',
 'achieve: 44',
 'achieved: 45',
 'achievement: 46',
 'achievements: 47',
 'achiever: 48',
 'achieving: 49',
 'acii: 50',
 'acquired: 51',
 'acquisition: 52',
 'acquisitions: 53',
 'act: 54',
 'acting: 55',
 'action: 56',
 'actions: 57',
 'actionscri

In [37]:
# generate vocabulary dict in format 'word_index: word'
voc_dict = {word.split(': ')[1]: word.split(': ')[0] for word in vocab}

In [38]:
voc_dict

{'0': 'aap',
 '1': 'aaron',
 '2': 'aat',
 '3': 'abb',
 '4': 'abenefit',
 '5': 'aberdeen',
 '6': 'abi',
 '7': 'abilities',
 '8': 'abreast',
 '9': 'abroad',
 '10': 'absence',
 '11': 'absolute',
 '12': 'ac',
 '13': 'aca',
 '14': 'academic',
 '15': 'academy',
 '16': 'acca',
 '17': 'accept',
 '18': 'acceptable',
 '19': 'acceptance',
 '20': 'accepted',
 '21': 'access',
 '22': 'accessible',
 '23': 'accident',
 '24': 'accommodates',
 '25': 'accommodation',
 '26': 'accomplished',
 '27': 'accordance',
 '28': 'account',
 '29': 'accountabilities',
 '30': 'accountability',
 '31': 'accountable',
 '32': 'accountancy',
 '33': 'accountant',
 '34': 'accountants',
 '35': 'accounting',
 '36': 'accounts',
 '37': 'accreditation',
 '38': 'accredited',
 '39': 'accruals',
 '40': 'accuracy',
 '41': 'accurate',
 '42': 'accurately',
 '43': 'achievable',
 '44': 'achieve',
 '45': 'achieved',
 '46': 'achievement',
 '47': 'achievements',
 '48': 'achiever',
 '49': 'achieving',
 '50': 'acii',
 '51': 'acquired',
 '52': 

In [39]:
'''
the doc_wordweights function takes the tfidf document vector file, as well as the w_index:word dictionary, 
creates the mapping between w_index and the actual word, and creates a dictionary of word:weight or each 
unique word appear in the document.
'''

def doc_wordweights(fName_tVectors, voc_dict):
    tfidf_weights = [] # a list to store the word:weight dictionaries of the document 
    
    with open(fName_tVectors) as tVecf:
        tVectors = tVecf.read().splitlines()
        
    for tv in tVectors: # for each tfidf document vector
        tv = tv.strip()
        weights = tv.split(' ') # list of word:weight entries
        weights = [w.split(':') for w in weights] # change the format of weight to the [word_index, weight] entries
        wordweight_dict = {voc_dict[w[0]]: w[1] for w in weights}
        tfidf_weights.append(wordweight_dict)
        
    return tfidf_weights

In [40]:
# generate the dictionary of word:weight for word appearing in the document
tfidf_weights = doc_wordweights('description_tVector.txt', voc_dict)

In [41]:
# investigate the word weights dictionary of the example job description
tfidf_weights[emp]


{'worked': '0.12521586951810817',
 'time': '0.07180883531936148',
 'term': '0.2180149197496349',
 'temporary': '0.10728635199658977',
 'processing': '0.11147392467381238',
 'process': '0.16017321954033129',
 'previously': '0.14329353805459263',
 'pm': '0.11021444536157937',
 'permanent': '0.08258696978692633',
 'payroll': '0.674806348886117',
 'monday': '0.11561862027395899',
 'long': '0.22294784934762477',
 'hours': '0.07699148329935733',
 'hempstead': '0.17511315143537287',
 'hemel': '0.17511315143537287',
 'happy': '0.14142427916139885',
 'friday': '0.11147392467381238',
 'european': '0.3927398409337309',
 'established': '0.09204887551248099',
 'employees': '0.24839900694149414'}

In [42]:
"""
construct the tf-idf weighted document embeddings. 

the following gen_docVecs function takes the word embeddings dictionary, the tokenized text of descriptions, 
and the tfidf weights (list of word:weight dictionaries, one for each description) as arguments, 
and generates the document embeddings
"""
def weighted_docvecs(embeddings, tfidf, docs):
    vecs = np.zeros((len(docs), embeddings.vector_size))
    for i, doc in enumerate(docs):
        valid_keys = [term for term in doc if term in embeddings.key_to_index]
        tf_weights = [float(tfidf[i].get(term, 0.)) for term in valid_keys]
        assert len(valid_keys) == len(tf_weights)
        weighted = [embeddings[term] * w for term, w in zip(valid_keys, tf_weights)]
        docvec = np.vstack(weighted)
        """
        Note: using `sum` here, other 'pooling' options are possible too,
        e.g. mean, etc.
        """
        docvec = np.sum(docvec, axis=0)
        vecs[i,:] = docvec
    return vecs

weighted_desw2v_dvs = weighted_docvecs(desw2v_wv, tfidf_weights, tk_descriptions)

#### Create Dataframe to store information of job data

In [43]:
# read targets from text file
with open('targets.txt', 'r') as f:
    targets = f.read().splitlines()

In [44]:
# create DataFrame to store the job data information
df = pd.DataFrame()
df['Targets'] = targets

In [45]:
df

,Targets
0,0
1,0
2,2
3,0
4,2
...,...
771,3
772,1
773,3
774,2


In [46]:
# add tokenised  descriptions to the dataframe
df['Tokenised descriptions'] = tk_descriptions

In [47]:
df.head(10)

,Targets,Tokenised descriptions
0,0,"[accountant, partqualified, south, east, londo..."
1,0,"[hedge, funds, london, recruiting, fund, accou..."
2,2,"[exciting, arisen, establish, provider, elderl..."
3,0,"[expanding, recruiting, junior, trainee, broke..."
4,2,"[rgn, nurses, hospitals, fulltime, part, swiis..."
5,1,"[production, coordinator, sandbach, pound, nda..."
6,2,"[reference, vac, title, scrub, nurse, location..."
7,0,"[recruit, experienced, purchase, ledger, clerk..."
8,3,"[west, suffolk, north, essex, cambridgeshire, ..."
9,3,"[executive, field, dartford, kbasic, ote, car,..."


In [48]:
# add Categories column based on targets column
df['Categories'] = df['Targets']

In [49]:
df.head(10)

,Targets,Tokenised descriptions,Categories
0,0,"[accountant, partqualified, south, east, londo...",0
1,0,"[hedge, funds, london, recruiting, fund, accou...",0
2,2,"[exciting, arisen, establish, provider, elderl...",2
3,0,"[expanding, recruiting, junior, trainee, broke...",0
4,2,"[rgn, nurses, hospitals, fulltime, part, swiis...",2
5,1,"[production, coordinator, sandbach, pound, nda...",1
6,2,"[reference, vac, title, scrub, nurse, location...",2
7,0,"[recruit, experienced, purchase, ledger, clerk...",0
8,3,"[west, suffolk, north, essex, cambridgeshire, ...",3
9,3,"[executive, field, dartford, kbasic, ote, car,...",3


In [50]:
# replace the target in categories column with categories
replace_dict = {'0': 'Accounting_Finance', '1': 'Engineering', '2': 'Healthcare_Nursing', '3': 'Sales'}
df['Categories'] = df['Categories'].replace(replace_dict)
df.head(10)

,Targets,Tokenised descriptions,Categories
0,0,"[accountant, partqualified, south, east, londo...",Accounting_Finance
1,0,"[hedge, funds, london, recruiting, fund, accou...",Accounting_Finance
2,2,"[exciting, arisen, establish, provider, elderl...",Healthcare_Nursing
3,0,"[expanding, recruiting, junior, trainee, broke...",Accounting_Finance
4,2,"[rgn, nurses, hospitals, fulltime, part, swiis...",Healthcare_Nursing
5,1,"[production, coordinator, sandbach, pound, nda...",Engineering
6,2,"[reference, vac, title, scrub, nurse, location...",Healthcare_Nursing
7,0,"[recruit, experienced, purchase, ledger, clerk...",Accounting_Finance
8,3,"[west, suffolk, north, essex, cambridgeshire, ...",Sales
9,3,"[executive, field, dartford, kbasic, ote, car,...",Sales


## Task 3. Job Advertisement Classification

In [51]:
seed = 0 # set a seed to make sure the experiment is reproducible 

### 3.1 Classification model based on different feature representations
Build the logistic regression model for document classification and report the model performance.

#### Count Vector for job descriptions

In [52]:
X_train, X_test, y_train, y_test = train_test_split(count_features, df['Categories'], test_size=0.33, random_state=seed)
model = LogisticRegression(max_iter=2000, random_state=seed)
model.fit(X_train, y_train)
print("Accuracy of count vector representation:", model.score(X_test, y_test))

Accuracy of count vector representation: 0.8715953307392996


#### Unweighted Word2vec based on job descriptions

In [53]:
# creating training and test split for unweighted in-house trained Word2vec feature representations

X_train, X_test, y_train, y_test,train_indices,test_indices = train_test_split(descriptionW2v_dvs, df['Categories'], list(range(0,len(df))),test_size=0.33, random_state=seed)

model = LogisticRegression(max_iter = 2000,random_state=seed)
model.fit(X_train, y_train)
# model.score(X_test, y_test)
print("Accuracy of unweighted Word2vec feature representation:", model.score(X_test, y_test))

Accuracy of unweighted Word2vec feature representation: 0.8287937743190662


C:\Users\nnkl2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### Weighted Word2vec based on job descriptions

In [54]:
# create training and test split for tf-idf weighted Word2vec feature representation
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(weighted_desw2v_dvs, df['Categories'], list(range(0, len(df))), test_size=0.33,random_state=seed)

model = LogisticRegression(max_iter= 2000, random_state=seed)
model.fit(X_train, y_train)
print("Accuracy of tf-idf weighted Word2vec feature representation: ", model.score(X_test, y_test))

Accuracy of tf-idf weighted Word2vec feature representation:  0.7392996108949417


In [55]:
num_folds = 5
# define the 5-fold cross validation
kf = KFold(n_splits = num_folds, random_state=seed, shuffle=True)
print(kf)
     

KFold(n_splits=5, random_state=0, shuffle=True)


In [56]:
# define function to build Logistic Regression model
def evaluate(X_train, X_test, y_train, y_test, seed):
    model = LogisticRegression(random_state=seed)
    model.fit(X_train, y_train)
    return model.score(X_test, y_test)

In [57]:
# conduct the cross validation and write the scores to dataframe
num_models=3
cv_df = pd.DataFrame(columns=['Count based on descriptions', 'Unweighted Word2vec based on descriptions', 'TF-IDF weighted Word2vec based on descriptions'], index=range(num_folds))

categories = df['Categories']
fold = 0 
for train_index, test_index in kf.split(list(range(0, len(categories)))):
    y_train = [categories[i] for i in train_index]
    y_test = [categories[i] for i in test_index]

    X_train_weighted, X_test_weigted = descriptionW2v_dvs[train_index], descriptionW2v_dvs[test_index]
    cv_df.loc[fold,'Unweighted Word2vec based on descriptions'] = evaluate(descriptionW2v_dvs[train_index],descriptionW2v_dvs[test_index],y_train,y_test,seed)
    
    X_train_count, X_test_count = count_features[train_index], count_features[test_index]
    cv_df.loc[fold,'Count based on descriptions'] = evaluate(count_features[train_index],count_features[test_index],y_train,y_test,seed)

    X_train_tfidf, X_test_tfidf = weighted_desw2v_dvs[train_index], weighted_desw2v_dvs[test_index]
    cv_df.loc[fold,'TF-IDF weighted Word2vec based on descriptions'] = evaluate(weighted_desw2v_dvs[train_index],weighted_desw2v_dvs[test_index],y_train,y_test,seed)
    
    fold +=1

C:\Users\nnkl2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\nnkl2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [58]:
cv_df

,Count based on descriptions,Unweighted Word2vec based on descriptions,TF-IDF weighted Word2vec based on descriptions
0,0.852564,0.685897,0.685897
1,0.883871,0.696774,0.709677
2,0.890323,0.787097,0.8
3,0.870968,0.716129,0.703226
4,0.922581,0.735484,0.709677


### 3.2 Test on different sets of information


##### 3.2.1. On only titles

In [59]:
"""
Preprocess the title
"""
job_data['data'][emp]

b'Title: European Payroll\nWebindex: 70757932\nCompany: Brookstreet UK\nDescription: Well established company based in Hemel Hempstead looking for someone to join the payroll team to process the payroll for their European employees Have you previously worked within a role where you have been processing European payroll? Are you available long term? Are you happy to work full time hours? This position will be Monday to Friday working ****:30am to ****:30pm. The role will be working within the payroll team to process payroll for their European employees. This is a temporary to permanent position so you must be available long term'

In [60]:
title = []
noneType=[]
for i in range(len(job_data['data'])):
    title_match =  re.search(r'Title:\s(.*)\n', job_data['data'][i].decode('utf-8'))
    if title_match:
        title.append(title_match.group(1))
    else: 
        noneType.append(i)
        
print(noneType)
print(len(title))
print(title[0])
print(type(title[emp]))

[]
776
Finance / Accounts Asst Bromley to ****k
<class 'str'>


In [61]:
def stats_print(tk_data):
    words = list(chain.from_iterable(tk_data)) # we put all the tokens in the corpus in a single list
    vocab = set(words) # compute the vocabulary by converting the list of words/tokens to a set, i.e., giving a set of unique words
    lexical_diversity = len(vocab)/len(words)
    print("Vocabulary size: ",len(vocab))
    print("Total number of tokens: ", len(words))
    print("Lexical diversity: ", lexical_diversity)
    print("Total number of datas:", len(tk_data))
    lens = [len(article) for article in tk_data]
    print("Average description length:", np.mean(lens))
    print("Maximum description length:", np.max(lens))
    print("Minimum description length:", np.min(lens))
    print("Standard deviation of data length:", np.std(lens))


In [62]:
from nltk import RegexpTokenizer
from nltk.tokenize import sent_tokenize
from itertools import chain 

def tokenizeData(orig_data):
    lower_des = orig_data.lower()
    
    # segment lowered description to sentences
    sentences = sent_tokenize(lower_des)
    
    #tokenize each sentence
    pattern = r"[a-zA-Z]+(?:[-'][a-zA-Z]+)?"
    
    tokenizer = RegexpTokenizer(pattern)
    token_lists = [tokenizer.tokenize(sen) for sen in sentences]
    
    # merge them into a list of tokens
    tokenised_data = list(chain.from_iterable(token_lists))
    return tokenised_data

In [63]:
# get the stopwords list
stopwords_file = './stopwords_en.txt'
with open(stopwords_file, 'r') as file: 
    stopwords_en = file.read(). splitlines()
    
len(stopwords_en)

571

In [64]:
def removeMostFreqWords(data, doc_fd):
    return [w for w in data if w not in doc_fd]

In [65]:
# function to preprocess a set of datas
def preprocess(datas):
    tk_datas = [tokenizeData(data) for data in datas]
    print(stats_print(tk_datas))
    
    # remove tokens with length < 2
    tk_datas = [[w for w in data if len(w) >=2] \
                   for data in tk_datas]
    print(stats_print(tk_datas))
    
    # remove tokens included in the stopwords
    tk_datas = [[w for w in data if w not in stopwords_en] \
                for data in tk_datas]
    print(stats_print(tk_datas))
    
    copy = tk_datas.copy()
    
    # remove less frequent words 
       
    words = list(chain.from_iterable(tk_datas))
    term_fd = FreqDist(words)
    lessFreqWords = set(term_fd.hapaxes())
    tk_datas = [[w for w in data if w not in lessFreqWords] \
                 for data in tk_datas]
    print(stats_print(tk_datas))
    
#   remove the most frequent words based on document frequency
      
    words_2 = list(chain.from_iterable([set(data) for data in copy]))
    doc_fd = FreqDist(words_2)
    df_words = [w[0] for w in doc_fd.most_common(50)]
    tk_datas = [removeMostFreqWords(data, df_words) for data in tk_datas]
    print(stats_print(tk_datas))
     
    return tk_datas

In [66]:
tk_titles = preprocess(title)

Vocabulary size:  1003
Total number of tokens:  3157
Lexical diversity:  0.3177066835603421
Total number of datas: 776
Average description length: 4.068298969072165
Maximum description length: 13
Minimum description length: 1
Standard deviation of data length: 1.8386529115562282
None
Vocabulary size:  985
Total number of tokens:  3077
Lexical diversity:  0.32011699707507313
Total number of datas: 776
Average description length: 3.9652061855670104
Maximum description length: 12
Minimum description length: 1
Standard deviation of data length: 1.7350467799519218
None
Vocabulary size:  954
Total number of tokens:  2963
Lexical diversity:  0.32197097536280794
Total number of datas: 776
Average description length: 3.818298969072165
Maximum description length: 10
Minimum description length: 1
Standard deviation of data length: 1.5653217426587334
None
Vocabulary size:  363
Total number of tokens:  2372
Lexical diversity:  0.153035413153457
Total number of datas: 776
Average description length:

In [67]:
stats_print(tk_titles)

Vocabulary size:  313
Total number of tokens:  1077
Lexical diversity:  0.2906220984215413
Total number of datas: 776
Average description length: 1.3878865979381443
Maximum description length: 6
Minimum description length: 0
Standard deviation of data length: 1.2047191003928313


In [68]:
def save_vocab(vocabFileName, vocab):
    out_file = open(vocabFileName, 'w') # creates a txt file and open to save the reviews
    string = "\n".join(word_string for word_string in vocab)
    out_file.write(string)
    out_file.close() # close the file

In [69]:
words = list(chain.from_iterable(tk_titles))
titles_vocab= sorted(list(set(words)))

# for i in range(len(titles_vocab)):
#     titles_vocab[i] = titles_vocab[i] + ": " + str(i)
    
# titles_vocab[emp]

In [70]:
titles_vocab  


['aberdeen',
 'accounting',
 'administration',
 'advertising',
 'advisor',
 'agency',
 'air',
 'allied',
 'analysis',
 'applications',
 'architect',
 'area',
 'asbestos',
 'ashford',
 'assistants',
 'associate',
 'assurance',
 'auditor',
 'automation',
 'ax',
 'band',
 'bangor',
 'bank',
 'barchester',
 'benefits',
 'bexhillonsea',
 'bias',
 'birmingham',
 'bookkeeper',
 'borer',
 'bournemouth',
 'branch',
 'bristol',
 'brokers',
 'bromley',
 'building',
 'buyer',
 'cad',
 'cambridge',
 'car',
 'cardiff',
 'carer',
 'cashier',
 'catering',
 'central',
 'centre',
 'chelmsford',
 'chichester',
 'children',
 'city',
 'claims',
 'clerk',
 'client',
 'cnc',
 'commissioning',
 'community',
 'company',
 'compliance',
 'conditioning',
 'construction',
 'contract',
 'contracts',
 'control',
 'controls',
 'corporate',
 'council',
 'credit',
 'customer',
 'data',
 'days',
 'db',
 'defence',
 'derby',
 'developer',
 'digital',
 'director',
 'document',
 'domiciliary',
 'dynamics',
 'edition',
 'ed

In [71]:
# print(len(vocab))
# vocab = [element.split(':')[0] for element in vocab] # extract the word only from vocab.txt 
# print(len(vocab))


In [72]:
titles_vocab[emp]

'area'

In [73]:
len(titles_vocab)

313

#### Count vector representation based on @title only 


In [74]:
cVectorizer = CountVectorizer(analyzer="word", vocabulary=titles_vocab) # initialise the CountVectorizer

In [75]:
title_count_features = cVectorizer.fit_transform([' '.join(title) for title in tk_titles]) # generate the count vector representation for all descriptions
print(title_count_features.shape)

(776, 313)


In [76]:
# investigate the count_features
title_count_features[emp]

<1x313 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

#### Unweighted vector represention based on @title only

In [77]:
sentences = tk_titles
title_w2v = gensim.models.Word2Vec(sentences=sentences, vector_size=100)

In [78]:
print(title_w2v)

Word2Vec<vocab=69, vector_size=100, alpha=0.025>


In [79]:
titlew2v_wv = title_w2v.wv

In [80]:
len(titlew2v_wv)

69

In [81]:
# function to generate document embeddings based on word embeddings
def docvecs(embeddings, docs):
    vecs = np.zeros((len(docs), embeddings.vector_size))
    for i, doc in enumerate(docs):
        valid_keys = [term for term in doc if term in embeddings.key_to_index]
        if valid_keys:  # handle empty valid_keys with job descriptions that have no titles
            docvec = np.vstack([embeddings[term] for term in valid_keys])
            """
            Note: using `sum` here, other 'pooling' options are possible too,
            e.g. mean, etc.
            """
            docvec = np.sum(docvec, axis=0)
            vecs[i,:] = docvec
        else: 
            pass
    return vecs


In [82]:
# generate unweighted document embeddings
titleW2v_dvs = docvecs(titlew2v_wv, tk_titles)

#### Weighted (TF-IDF Weighted) vector representation @title only

In [83]:
# generate tf-idf vectors
from sklearn.feature_extraction.text import TfidfVectorizer
tVectorizer = TfidfVectorizer(analyzer = "word",vocabulary = titles_vocab) # initialised the TfidfVectorizer
tfidf_features = tVectorizer.fit_transform([' '.join(title) for title in tk_titles]) # generate the tfidf vector representation for all titles
write_vectorFile(tfidf_features, 'title_tVector.txt')

In [84]:
titles_vocab

['aberdeen',
 'accounting',
 'administration',
 'advertising',
 'advisor',
 'agency',
 'air',
 'allied',
 'analysis',
 'applications',
 'architect',
 'area',
 'asbestos',
 'ashford',
 'assistants',
 'associate',
 'assurance',
 'auditor',
 'automation',
 'ax',
 'band',
 'bangor',
 'bank',
 'barchester',
 'benefits',
 'bexhillonsea',
 'bias',
 'birmingham',
 'bookkeeper',
 'borer',
 'bournemouth',
 'branch',
 'bristol',
 'brokers',
 'bromley',
 'building',
 'buyer',
 'cad',
 'cambridge',
 'car',
 'cardiff',
 'carer',
 'cashier',
 'catering',
 'central',
 'centre',
 'chelmsford',
 'chichester',
 'children',
 'city',
 'claims',
 'clerk',
 'client',
 'cnc',
 'commissioning',
 'community',
 'company',
 'compliance',
 'conditioning',
 'construction',
 'contract',
 'contracts',
 'control',
 'controls',
 'corporate',
 'council',
 'credit',
 'customer',
 'data',
 'days',
 'db',
 'defence',
 'derby',
 'developer',
 'digital',
 'director',
 'document',
 'domiciliary',
 'dynamics',
 'edition',
 'ed

In [85]:
titles_vocab = [str(i) + ": " + titles_vocab[i] for i in range(len(titles_vocab))]
titles_vocab

['0: aberdeen',
 '1: accounting',
 '2: administration',
 '3: advertising',
 '4: advisor',
 '5: agency',
 '6: air',
 '7: allied',
 '8: analysis',
 '9: applications',
 '10: architect',
 '11: area',
 '12: asbestos',
 '13: ashford',
 '14: assistants',
 '15: associate',
 '16: assurance',
 '17: auditor',
 '18: automation',
 '19: ax',
 '20: band',
 '21: bangor',
 '22: bank',
 '23: barchester',
 '24: benefits',
 '25: bexhillonsea',
 '26: bias',
 '27: birmingham',
 '28: bookkeeper',
 '29: borer',
 '30: bournemouth',
 '31: branch',
 '32: bristol',
 '33: brokers',
 '34: bromley',
 '35: building',
 '36: buyer',
 '37: cad',
 '38: cambridge',
 '39: car',
 '40: cardiff',
 '41: carer',
 '42: cashier',
 '43: catering',
 '44: central',
 '45: centre',
 '46: chelmsford',
 '47: chichester',
 '48: children',
 '49: city',
 '50: claims',
 '51: clerk',
 '52: client',
 '53: cnc',
 '54: commissioning',
 '55: community',
 '56: company',
 '57: compliance',
 '58: conditioning',
 '59: construction',
 '60: contract',

In [86]:
# generate vocabulary dict in format 'word_index: word'
voc_dict = {word.split(': ')[0]: word.split(': ')[1] for word in titles_vocab}

In [87]:
voc_dict

{'0': 'aberdeen',
 '1': 'accounting',
 '2': 'administration',
 '3': 'advertising',
 '4': 'advisor',
 '5': 'agency',
 '6': 'air',
 '7': 'allied',
 '8': 'analysis',
 '9': 'applications',
 '10': 'architect',
 '11': 'area',
 '12': 'asbestos',
 '13': 'ashford',
 '14': 'assistants',
 '15': 'associate',
 '16': 'assurance',
 '17': 'auditor',
 '18': 'automation',
 '19': 'ax',
 '20': 'band',
 '21': 'bangor',
 '22': 'bank',
 '23': 'barchester',
 '24': 'benefits',
 '25': 'bexhillonsea',
 '26': 'bias',
 '27': 'birmingham',
 '28': 'bookkeeper',
 '29': 'borer',
 '30': 'bournemouth',
 '31': 'branch',
 '32': 'bristol',
 '33': 'brokers',
 '34': 'bromley',
 '35': 'building',
 '36': 'buyer',
 '37': 'cad',
 '38': 'cambridge',
 '39': 'car',
 '40': 'cardiff',
 '41': 'carer',
 '42': 'cashier',
 '43': 'catering',
 '44': 'central',
 '45': 'centre',
 '46': 'chelmsford',
 '47': 'chichester',
 '48': 'children',
 '49': 'city',
 '50': 'claims',
 '51': 'clerk',
 '52': 'client',
 '53': 'cnc',
 '54': 'commissioning',
 

In [88]:
'''
the doc_wordweights function takes the tfidf document vector file, as well as the w_index:word dictionary, 
creates the mapping between w_index and the actual word, and creates a dictionary of word:weight or each 
unique word appear in the document.
'''
def doc_wordweights(fName_tVectors, voc_dict):
    tfidf_weights = [] # a list to store the word:weight dictionaries of the document 
    
    with open(fName_tVectors) as tVecf:
        tVectors = tVecf.read().splitlines()
        
    for tv in tVectors: # for each tfidf document vector
        tv = tv.strip()
        weights = tv.split(' ') # list of word:weight entries
        weights = [w.split(':') for w in weights] # change the format of weight to the [word_index, weight] entries
         # Filter out entries with empty strings in word index or weight
        valid_weights = [(w[0], w[1]) for w in weights if len(w) == 2 and w[0] and w[1]]
        wordweight_dict = {voc_dict[w[0]]: w[1] for w in valid_weights}
        tfidf_weights.append(wordweight_dict)
        
    return tfidf_weights

In [89]:
# generate the dictionary of word:weight for word appearing in the document
title_tfidf_weights = doc_wordweights('./title_tVector.txt', voc_dict)

In [90]:
# investigate the word weights dictionary of the example job title
title_tfidf_weights[emp]

{'payroll': '0.6962003337095456', 'european': '0.7178475432448853'}

In [91]:
"""
construct the tf-idf weighted document embeddings. 

the following gen_docVecs function takes the word embeddings dictionary, the tokenized text of descriptions, 
and the tfidf weights (list of word:weight dictionaries, one for each description) as arguments, 
and generates the document embeddings
"""
def weighted_docvecs(embeddings, tfidf, docs):
    vecs = np.zeros((len(docs), embeddings.vector_size))
    for i, doc in enumerate(docs):
        valid_keys = [term for term in doc if term in embeddings.key_to_index]
        tf_weights = [float(tfidf[i].get(term, 0.)) for term in valid_keys]
        assert len(valid_keys) == len(tf_weights)
        # if there are valid keys and weights
        if valid_keys and tf_weights:
            weighted = [embeddings[term] * w for term, w in zip(valid_keys, tf_weights)]
            docvec = np.vstack(weighted)
            
            # if docvec is not empty before summing
            if docvec.size > 0:
                docvec = np.sum(docvec, axis=0)
                vecs[i, :] = docvec
            else:
                # Handle the case where docvec is empty (no valid embeddings)
                vecs[i, :] = np.zeros(embeddings.vector_size)
        else:
            # Handle the case where there are no valid keys or weights
            vecs[i, :] = np.zeros(embeddings.vector_size)
    return vecs

weighted_titlew2v_dvs = weighted_docvecs(titlew2v_wv, title_tfidf_weights, tk_titles)

#### update the Dataframe with tokenised titles

In [92]:
# add tokenised  descriptions to the dataframe
df['Tokenised titles'] = tk_titles

In [93]:
df.head(10)

,Targets,Tokenised descriptions,Categories,Tokenised titles
0,0,"[accountant, partqualified, south, east, londo...",Accounting_Finance,[bromley]
1,0,"[hedge, funds, london, recruiting, fund, accou...",Accounting_Finance,"[fund, fund]"
2,2,"[exciting, arisen, establish, provider, elderl...",Healthcare_Nursing,[]
3,0,"[expanding, recruiting, junior, trainee, broke...",Accounting_Finance,[brokers]
4,2,"[rgn, nurses, hospitals, fulltime, part, swiis...",Healthcare_Nursing,[nurses]
5,1,"[production, coordinator, sandbach, pound, nda...",Engineering,[production]
6,2,"[reference, vac, title, scrub, nurse, location...",Healthcare_Nursing,[scrub]
7,0,"[recruit, experienced, purchase, ledger, clerk...",Accounting_Finance,"[purchase, ledger, clerk, maternity]"
8,3,"[west, suffolk, north, essex, cambridgeshire, ...",Sales,[]
9,3,"[executive, field, dartford, kbasic, ote, car,...",Sales,[]


#### Classification model based on different feature representations

In [94]:
categories = df['Categories']
fold = 0 
for train_index, test_index in kf.split(list(range(0, len(categories)))):
    y_train = [categories[i] for i in train_index]
    y_test = [categories[i] for i in test_index]
    
    X_train_count, X_test_count = title_count_features[train_index], title_count_features[test_index]
    cv_df.loc[fold,'Count features based on titles'] = evaluate(title_count_features[train_index],title_count_features[test_index],y_train,y_test,seed)

    X_train_weighted, X_test_weighted = titleW2v_dvs[train_index], titleW2v_dvs[test_index]
    cv_df.loc[fold,'Unweighted Word2vec based on titles'] = evaluate(titleW2v_dvs[train_index],titleW2v_dvs[test_index],y_train,y_test,seed)
    
    X_train_tfidf, X_test_tfidf = weighted_titlew2v_dvs[train_index], weighted_titlew2v_dvs[test_index]
    cv_df.loc[fold,'TF-IDF weighted Word2vec based on titles'] = evaluate(weighted_titlew2v_dvs[train_index],weighted_titlew2v_dvs[test_index],y_train,y_test,seed)
    
    fold +=1

In [95]:
cv_df

,Count based on descriptions,Unweighted Word2vec based on descriptions,TF-IDF weighted Word2vec based on descriptions,Count features based on titles,Unweighted Word2vec based on titles,TF-IDF weighted Word2vec based on titles
0,0.852564,0.685897,0.685897,0.653846,0.282051,0.282051
1,0.883871,0.696774,0.709677,0.606452,0.316129,0.316129
2,0.890323,0.787097,0.8,0.567742,0.296774,0.296774
3,0.870968,0.716129,0.703226,0.645161,0.303226,0.303226
4,0.922581,0.735484,0.709677,0.593548,0.290323,0.290323


In [96]:
# retrieve the description_data from task1
%store -r description_data

In [97]:
len(description_data)

776

In [98]:
len(title)

776

In [99]:
# store both titles and descriptions in a mix list
mix = [title[i] + " " + description_data[i] for i in range(len(job_data['data']))]

In [100]:
mix[emp]

'European Payroll Well established company based in Hemel Hempstead looking for someone to join the payroll team to process the payroll for their European employees Have you previously worked within a role where you have been processing European payroll? Are you available long term? Are you happy to work full time hours? This position will be Monday to Friday working ****:30am to ****:30pm. The role will be working within the payroll team to process payroll for their European employees. This is a temporary to permanent position so you must be available long term'

In [101]:
tk_mix = preprocess(mix)

Vocabulary size:  9905
Total number of tokens:  191752
Lexical diversity:  0.0516552630481038
Total number of datas: 776
Average description length: 247.10309278350516
Maximum description length: 854
Minimum description length: 20
Standard deviation of data length: 126.90815613477123
None
Vocabulary size:  9879
Total number of tokens:  184847
Lexical diversity:  0.053444199797670504
Total number of datas: 776
Average description length: 238.2048969072165
Maximum description length: 813
Minimum description length: 20
Standard deviation of data length: 122.51178368209628
None
Vocabulary size:  9475
Total number of tokens:  110981
Lexical diversity:  0.08537497409466485
Total number of datas: 776
Average description length: 143.01675257731958
Maximum description length: 491
Minimum description length: 18
Standard deviation of data length: 74.04119762777529
None
Vocabulary size:  5298
Total number of tokens:  106804
Lexical diversity:  0.049604883712220514
Total number of datas: 776
Averag

In [102]:
words = list(chain.from_iterable(tk_mix))
mix_vocab= sorted(list(set(words)))

# for i in range(len(titles_vocab)):
#     titles_vocab[i] = titles_vocab[i] + ": " + str(i)
    
# titles_vocab[emp]

In [103]:
mix_vocab  

['aap',
 'aaron',
 'aat',
 'abb',
 'abenefit',
 'aberdeen',
 'abi',
 'abilities',
 'abreast',
 'abroad',
 'absence',
 'absolute',
 'ac',
 'aca',
 'academic',
 'academy',
 'acca',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'access',
 'accessible',
 'accident',
 'accommodates',
 'accommodation',
 'accomplished',
 'accordance',
 'accounant',
 'account',
 'accountabilities',
 'accountability',
 'accountable',
 'accountancy',
 'accountant',
 'accountants',
 'accounting',
 'accounts',
 'accreditation',
 'accredited',
 'accruals',
 'accuracy',
 'accurate',
 'accurately',
 'achievable',
 'achieve',
 'achieved',
 'achievement',
 'achievements',
 'achiever',
 'achieving',
 'acii',
 'acquired',
 'acquisition',
 'acquisitions',
 'act',
 'acting',
 'action',
 'actions',
 'actionscript',
 'active',
 'actively',
 'activites',
 'activities',
 'activity',
 'acts',
 'actual',
 'actuarial',
 'acumen',
 'acute',
 'ad',
 'adam',
 'adapt',
 'adaptability',
 'add',
 'added',
 'addiction',
 'addin

In [104]:
mix_vocab[emp]

'absolute'

In [105]:
len(mix_vocab)

5248

#### Count vector representation @titles and descriptions  


In [106]:
cVectorizer = CountVectorizer(analyzer="word", vocabulary=mix_vocab) # initialise the CountVectorizer

In [107]:
mix_count_features = cVectorizer.fit_transform([' '.join(data) for data in tk_mix]) # generate the count vector representation for all descriptions
print(mix_count_features.shape)

(776, 5248)


In [108]:
# investigate the count_features
mix_count_features[emp]

<1x5248 sparse matrix of type '<class 'numpy.int64'>'
	with 20 stored elements in Compressed Sparse Row format>

#### unweighted vector represention @titles and descriptions

In [109]:
sentences = tk_mix
mix_w2v = gensim.models.Word2Vec(sentences=sentences, vector_size=100)

In [110]:
print(mix_w2v)

Word2Vec<vocab=2806, vector_size=100, alpha=0.025>


In [111]:
mixw2v_wv = mix_w2v.wv

In [112]:
len(mixw2v_wv)

2806

In [113]:
mixW2v_dvs = docvecs(mixw2v_wv, tk_mix)

#### Weighted (TF-IDF Weighted) vector representation @titles and descriptions

In [114]:
# generate tf-idf vectors
from sklearn.feature_extraction.text import TfidfVectorizer
tVectorizer = TfidfVectorizer(analyzer = "word",vocabulary = mix_vocab) # initialised the TfidfVectorizer
tfidf_features = tVectorizer.fit_transform([' '.join(data) for data in tk_mix]) # generate the tfidf vector representation for all titles
write_vectorFile(tfidf_features, 'mix_tVector.txt')

In [115]:
mix_vocab

['aap',
 'aaron',
 'aat',
 'abb',
 'abenefit',
 'aberdeen',
 'abi',
 'abilities',
 'abreast',
 'abroad',
 'absence',
 'absolute',
 'ac',
 'aca',
 'academic',
 'academy',
 'acca',
 'accept',
 'acceptable',
 'acceptance',
 'accepted',
 'access',
 'accessible',
 'accident',
 'accommodates',
 'accommodation',
 'accomplished',
 'accordance',
 'accounant',
 'account',
 'accountabilities',
 'accountability',
 'accountable',
 'accountancy',
 'accountant',
 'accountants',
 'accounting',
 'accounts',
 'accreditation',
 'accredited',
 'accruals',
 'accuracy',
 'accurate',
 'accurately',
 'achievable',
 'achieve',
 'achieved',
 'achievement',
 'achievements',
 'achiever',
 'achieving',
 'acii',
 'acquired',
 'acquisition',
 'acquisitions',
 'act',
 'acting',
 'action',
 'actions',
 'actionscript',
 'active',
 'actively',
 'activites',
 'activities',
 'activity',
 'acts',
 'actual',
 'actuarial',
 'acumen',
 'acute',
 'ad',
 'adam',
 'adapt',
 'adaptability',
 'add',
 'added',
 'addiction',
 'addin

In [116]:
mix_vocab = [str(i) + ": " + mix_vocab[i] for i in range(len(mix_vocab))]
mix_vocab

['0: aap',
 '1: aaron',
 '2: aat',
 '3: abb',
 '4: abenefit',
 '5: aberdeen',
 '6: abi',
 '7: abilities',
 '8: abreast',
 '9: abroad',
 '10: absence',
 '11: absolute',
 '12: ac',
 '13: aca',
 '14: academic',
 '15: academy',
 '16: acca',
 '17: accept',
 '18: acceptable',
 '19: acceptance',
 '20: accepted',
 '21: access',
 '22: accessible',
 '23: accident',
 '24: accommodates',
 '25: accommodation',
 '26: accomplished',
 '27: accordance',
 '28: accounant',
 '29: account',
 '30: accountabilities',
 '31: accountability',
 '32: accountable',
 '33: accountancy',
 '34: accountant',
 '35: accountants',
 '36: accounting',
 '37: accounts',
 '38: accreditation',
 '39: accredited',
 '40: accruals',
 '41: accuracy',
 '42: accurate',
 '43: accurately',
 '44: achievable',
 '45: achieve',
 '46: achieved',
 '47: achievement',
 '48: achievements',
 '49: achiever',
 '50: achieving',
 '51: acii',
 '52: acquired',
 '53: acquisition',
 '54: acquisitions',
 '55: act',
 '56: acting',
 '57: action',
 '58: acti

In [117]:
# generate vocabulary dict in format 'word_index: word'
voc_dict = {word.split(': ')[0]: word.split(': ')[1] for word in mix_vocab}

In [118]:
voc_dict

{'0': 'aap',
 '1': 'aaron',
 '2': 'aat',
 '3': 'abb',
 '4': 'abenefit',
 '5': 'aberdeen',
 '6': 'abi',
 '7': 'abilities',
 '8': 'abreast',
 '9': 'abroad',
 '10': 'absence',
 '11': 'absolute',
 '12': 'ac',
 '13': 'aca',
 '14': 'academic',
 '15': 'academy',
 '16': 'acca',
 '17': 'accept',
 '18': 'acceptable',
 '19': 'acceptance',
 '20': 'accepted',
 '21': 'access',
 '22': 'accessible',
 '23': 'accident',
 '24': 'accommodates',
 '25': 'accommodation',
 '26': 'accomplished',
 '27': 'accordance',
 '28': 'accounant',
 '29': 'account',
 '30': 'accountabilities',
 '31': 'accountability',
 '32': 'accountable',
 '33': 'accountancy',
 '34': 'accountant',
 '35': 'accountants',
 '36': 'accounting',
 '37': 'accounts',
 '38': 'accreditation',
 '39': 'accredited',
 '40': 'accruals',
 '41': 'accuracy',
 '42': 'accurate',
 '43': 'accurately',
 '44': 'achievable',
 '45': 'achieve',
 '46': 'achieved',
 '47': 'achievement',
 '48': 'achievements',
 '49': 'achiever',
 '50': 'achieving',
 '51': 'acii',
 '52':

In [119]:
# generate the dictionary of word:weight for word appearing in the document
mix_tfidf_weights = doc_wordweights('./mix_tVector.txt', voc_dict)

In [120]:
# investigate the word weights dictionary of the example job description
mix_tfidf_weights[emp]

{'worked': '0.1092161571357959',
 'time': '0.06220872391991935',
 'term': '0.1891368743610237',
 'temporary': '0.09309641333979624',
 'processing': '0.09723011724131589',
 'process': '0.13970676066529508',
 'previously': '0.12498391481002581',
 'pm': '0.09613157045963792',
 'permanent': '0.07182802535076827',
 'payroll': '0.7062979143440351',
 'monday': '0.10084521593198298',
 'long': '0.19446023448263178',
 'hours': '0.06715373994128264',
 'hempstead': '0.1527377123787357',
 'hemel': '0.1527377123787357',
 'happy': '0.12335350427346829',
 'friday': '0.09723011724131589',
 'european': '0.45244161346182643',
 'established': '0.08028714324178043',
 'employees': '0.21665931865429067'}

In [121]:
weighted_mixw2v_dvs = weighted_docvecs(mixw2v_wv, mix_tfidf_weights, tk_mix)

#### update DataFrame with tokenised titles and descriptions

In [122]:
# add tokenised  descriptions to the dataframe
df['Tokenised title and descriptions'] = tk_mix

In [123]:
df.head(10)

,Targets,Tokenised descriptions,Categories,Tokenised titles,Tokenised title and descriptions
0,0,"[accountant, partqualified, south, east, londo...",Accounting_Finance,[bromley],"[finance, accounts, bromley, accountant, partq..."
1,0,"[hedge, funds, london, recruiting, fund, accou...",Accounting_Finance,"[fund, fund]","[fund, accountant, hedge, fund, hedge, funds, ..."
2,2,"[exciting, arisen, establish, provider, elderl...",Healthcare_Nursing,[],"[deputy, home, exciting, arisen, establish, pr..."
3,0,"[expanding, recruiting, junior, trainee, broke...",Accounting_Finance,[brokers],"[brokers, wanted, start, expanding, recruiting..."
4,2,"[rgn, nurses, hospitals, fulltime, part, swiis...",Healthcare_Nursing,[nurses],"[rgn, nurses, hospitals, penarth, rgn, nurses,..."
5,1,"[production, coordinator, sandbach, pound, nda...",Engineering,[production],"[production, coordinator, production, coordina..."
6,2,"[reference, vac, title, scrub, nurse, location...",Healthcare_Nursing,[scrub],"[scrub, nurse, reference, vac, title, scrub, n..."
7,0,"[recruit, experienced, purchase, ledger, clerk...",Accounting_Finance,"[purchase, ledger, clerk, maternity]","[purchase, ledger, clerk, maternity, cover, re..."
8,3,"[west, suffolk, north, essex, cambridgeshire, ...",Sales,[],"[executive, west, suffolk, north, essex, cambr..."
9,3,"[executive, field, dartford, kbasic, ote, car,...",Sales,[],"[executive, field, dartford, executive, field,..."


#### Classification model for @title and descriptions

In [124]:
cv_df

,Count based on descriptions,Unweighted Word2vec based on descriptions,TF-IDF weighted Word2vec based on descriptions,Count features based on titles,Unweighted Word2vec based on titles,TF-IDF weighted Word2vec based on titles
0,0.852564,0.685897,0.685897,0.653846,0.282051,0.282051
1,0.883871,0.696774,0.709677,0.606452,0.316129,0.316129
2,0.890323,0.787097,0.8,0.567742,0.296774,0.296774
3,0.870968,0.716129,0.703226,0.645161,0.303226,0.303226
4,0.922581,0.735484,0.709677,0.593548,0.290323,0.290323


In [125]:
categories = df['Categories']
fold = 0 
for train_index, test_index in kf.split(list(range(0, len(categories)))):
    y_train = [categories[i] for i in train_index]
    y_test = [categories[i] for i in test_index]

    X_train_count, X_test_count = mix_count_features[train_index], mix_count_features[test_index]
    cv_df.loc[fold,'Count features based on titles&descriptions'] = evaluate(mix_count_features[train_index],mix_count_features[test_index],y_train,y_test,seed)

    X_train_weighted, X_test_weighted = mixW2v_dvs[train_index], mixW2v_dvs[test_index]
    cv_df.loc[fold,'Unweighted Word2vec based on titles&descriptions'] = evaluate(mixW2v_dvs[train_index],mixW2v_dvs[test_index],y_train,y_test,seed)
    
    X_train_tfidf, X_test_tfidf = weighted_mixw2v_dvs[train_index], weighted_mixw2v_dvs[test_index]
    cv_df.loc[fold,'TF-IDF weighted Word2vec based on titles&descriptions'] = evaluate(weighted_mixw2v_dvs[train_index],weighted_mixw2v_dvs[test_index],y_train,y_test,seed)
    
    fold +=1

C:\Users\nnkl2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\nnkl2\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [126]:
cv_df

,Count based on descriptions,Unweighted Word2vec based on descriptions,TF-IDF weighted Word2vec based on descriptions,Count features based on titles,Unweighted Word2vec based on titles,TF-IDF weighted Word2vec based on titles,Count features based on titles&descriptions,Unweighted Word2vec based on titles&descriptions,TF-IDF weighted Word2vec based on titles&descriptions
0,0.852564,0.685897,0.685897,0.653846,0.282051,0.282051,0.858974,0.692308,0.737179
1,0.883871,0.696774,0.709677,0.606452,0.316129,0.316129,0.916129,0.748387,0.761290
2,0.890323,0.787097,0.8,0.567742,0.296774,0.296774,0.877419,0.832258,0.806452
3,0.870968,0.716129,0.703226,0.645161,0.303226,0.303226,0.883871,0.767742,0.729032
4,0.922581,0.735484,0.709677,0.593548,0.290323,0.290323,0.909677,0.780645,0.741935


It can be seen from the table that more information do not necessarily provides higher accuracy. However, it can be concluded that model build using descriptions data will provide higher accuracy compared to using only titles.

## Summary
- The tasks included in task 2&3 are a challenging way to practice knowledge relating to NL processing, classification and build models to classify job data based on categories. I have got chances to practice with real datasets and investigate the accuracy of models built to classfify the job data.